Chapter 7 Exercise 5  

- Write a query against the EmpYearOrders table that unpivots the data, returning a row for each employee and order year with the number of orders. Exclude rows in which the number of orders is 0 (in this example, employee 3 in the year 2015)

In [ ]:
USE TSQLV4
SELECT empid, CAST(RIGHT(orderyear, 4) AS INT) AS orderyear, numorders
FROM dbo.EmpYearOrders
UNPIVOT(numorders FOR orderyear IN(cnt2014, cnt2015, cnt2016)) AS U
WHERE numorders <> 0

1.  Identify the most expensive products within each sales order by assigning them a rank. 
For each SalesOrderID, this query ranks products from highest to lowest LineTotal. If two products within the same SalesOrderID have the same LineTotal, they’ll receive the same rank, and the next rank will skip a number.

In [ ]:
use AdventureWorks2017
SELECT 
    SalesOrderID, 
    ProductID, 
    LineTotal,
    RANK() OVER (PARTITION BY SalesOrderID ORDER BY LineTotal DESC) AS ProductRank
FROM Sales.SalesOrderDetail;


2.  Return a list of orders with their order dates, subtotals, and a RunningTotal column that shows the cumulative subtotal for all orders up to that date within each year.

In [ ]:
use AdventureWorks2017
SELECT 
    OrderDate, 
    SubTotal,
    SUM(SubTotal) OVER (PARTITION BY YEAR(OrderDate) ORDER BY OrderDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningTotal
FROM Sales.SalesOrderHeader;


3. Pivot sales data to create a table where each row shows the ProductSubcategoryID and its total sales in 2011, 2012, and 2013.
The PIVOT function takes the data from SourceTable and reorganizes it so that each OrderYear (2011, 2012, 2013) becomes a separate column.
SUM(LineTotal) aggregates the total sales (LineTotal) for each ProductSubcategoryID in each year.

In [ ]:
use AdventureWorks2017
SELECT ProductSubcategoryID, [2011], [2012], [2013]
FROM (
    SELECT 
        ps.ProductSubcategoryID, 
        YEAR(soh.OrderDate) AS OrderYear, 
        sod.LineTotal
    FROM 
        Sales.SalesOrderDetail sod
    JOIN 
        Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
    JOIN 
        Production.Product p ON sod.ProductID = p.ProductID
    JOIN 
        Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
) AS SourceTable
PIVOT (
    SUM(LineTotal) 
    FOR OrderYear IN ([2011], [2012], [2013])
) AS PivotedData;


4. Getting a breakdown of sales by territory, by year, and by territory-year combinations in one result set.

In [ ]:
use AdventureWorks2017
SELECT 
    TerritoryID,
    YEAR(OrderDate) AS OrderYear,
    SUM(TotalDue) AS TotalSales
FROM 
    Sales.SalesOrderHeader
GROUP BY 
    GROUPING SETS ((TerritoryID), (YEAR(OrderDate)), (TerritoryID, YEAR(OrderDate)));


5. A running total of sales (cumulative sales) for each order, where the cumulative value increases as each new order is added, sorted by OrderDate. This is to track the total sales over time.

In [ ]:
use AdventureWorks2017
SELECT 
    SalesOrderID,
    OrderDate,
    TotalDue,
    SUM(TotalDue) OVER (ORDER BY OrderDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS CumulativeSales
FROM 
    Sales.SalesOrderHeader;


6.  Provide a breakdown of sales for each product by year, along with the sales from the previous year and the change in sales (growth or decline) compared to the previous year.

In [ ]:
use AdventureWorks2017
SELECT 
    ProductID,
    YEAR(OrderDate) AS OrderYear,
    SUM(LineTotal) AS YearlySales,
    LAG(SUM(LineTotal)) OVER (PARTITION BY ProductID ORDER BY YEAR(OrderDate)) AS PreviousYearSales,
    SUM(LineTotal) - LAG(SUM(LineTotal)) OVER (PARTITION BY ProductID ORDER BY YEAR(OrderDate)) AS SalesGrowth
FROM 
    Sales.SalesOrderDetail sod
JOIN 
    Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
GROUP BY 
    ProductID, YEAR(OrderDate);


7\. Project total sales for specific territories, for specific years, and overall.<span style="color: var(--vscode-foreground);">The </span> `ROLLUP` <span style="color: var(--vscode-foreground);"> operator creates multiple summary levels:</span>

- First, it calculates sales for each `TerritoryID` and `OrderYear`.
- Then, it calculates subtotal sales for each territory across all years.
- Finally, it calculates the grand total sales across all territories and years.

In [ ]:
use AdventureWorks2017
SELECT 
    TerritoryID,
    YEAR(OrderDate) AS OrderYear,
    SUM(TotalDue) AS TotalSales
FROM 
    Sales.SalesOrderHeader
GROUP BY 
    ROLLUP (TerritoryID, YEAR(OrderDate));


8. Rank each sales order for each year based on the TotalDue amount. The highest sales order amount for each year gets a rank of 1, and the rank number increases for the lower amounts. If two orders have the same TotalDue, they will share the same rank, and the next order will have the next available rank number.

In [ ]:
use AdventureWorks2017
SELECT 
    SalesOrderID,
    YEAR(OrderDate) AS OrderYear,
    TotalDue,
    RANK() OVER (PARTITION BY YEAR(OrderDate) ORDER BY TotalDue DESC) AS SalesRank
FROM 
    Sales.SalesOrderHeader;


9. Calculates how each sales order ranks as a percentage of total sales within each sales territory. The rank is based on the TotalDue value, where the highest TotalDue within each territory gets the highest percentage rank (closer to 1) and the lowest gets the lowest rank (closer to 0). It helps understand where each sales order stands within its territory in terms of total sales.

In [ ]:
use AdventureWorks2017
SELECT 
    TerritoryID,
    SalesOrderID,
    TotalDue,
    PERCENT_RANK() OVER (PARTITION BY TerritoryID ORDER BY TotalDue DESC) AS SalesPercentRank
FROM 
    Sales.SalesOrderHeader
WHERE 
    TerritoryID IS NOT NULL;


10\. This query calculates the **total sales for each month** in the year 2013. It groups the sales orders by month, sums the sales for each month, and then displays the total sales for each month from January to December. The results are ordered by month, with the earliest month (January) appearing first.

In [ ]:
use AdventureWorks2017
SELECT 
    DATEPART(MONTH, OrderDate) AS OrderMonth,
    SUM(TotalDue) AS MonthlySales
FROM 
    Sales.SalesOrderHeader
WHERE 
    YEAR(OrderDate) = 2013
GROUP BY 
    DATEPART(MONTH, OrderDate)
ORDER BY 
    OrderMonth;
